In [2]:
import pandas as pd
import numpy as np

In [9]:
# def input_dataframe() -> pd.DataFrame:
#     num_cols = 500
#     num_electrodes = 10
#     elec_data = np.random.rand(num_electrodes, num_cols)
#     cols = [i for i in range(num_cols)]
#     return pd.DataFrame(elec_data, columns=cols)

in_df = pd.read_parquet("/Users/siyuanzhao/Documents/DATADIR/patients/ep187/sample1/eegData_fast.parquet", engine="pyarrow")
in_df.head(11)
in_df
# in_df_test = in_df[in_df["electrode"] == 4 & (in_df["ms"] >= 0 & in_df["ms"] <= 500)]
# in_df_test = in_df[(in_df["electrode"] == 4) & (in_df["ms"] >= 0) & (in_df["ms"] <= 500)]
# electrode_list =[1,2,3]
# filtered_df = in_df[in_df['electrode'].isin(electrode_list)]

# filtered_df


,0,1,2,3,4,5,6,7,8,9,...,515062,515063,515064,515065,515066,515067,515068,515069,515070,515071
0,-63.478497,-61.189610,-56.001461,-53.865166,-48.982204,-41.962948,-31.128878,-22.583696,-15.564439,-11.597033,...,641.804199,622.577576,579.241272,518.967224,453.657623,391.857635,337.839874,295.266571,268.105103,256.813232
1,75.533310,73.397011,72.634048,73.702194,73.702194,72.328865,70.039978,69.887383,70.345161,69.582199,...,46.082947,5.035554,-43.946651,-94.302193,-139.011810,-175.634018,-204.473999,-221.716965,-221.259186,-200.964371
2,50.202946,45.472576,37.232578,25.482954,15.564439,7.171849,1.220740,1.678518,7.019257,11.444441,...,164.647354,153.813278,125.736252,85.909599,43.641468,7.171849,-18.311104,-29.297768,-23.499250,-1.678518
3,82.857750,86.519974,89.724411,90.334785,88.808861,87.740707,84.536270,80.263680,76.143677,72.176270,...,188.756973,171.819199,137.180695,90.792564,39.979244,-10.834070,-57.527386,-93.691818,-111.392555,-105.746628
4,32.502213,37.690357,42.268135,48.524429,53.102203,58.137756,59.816277,62.105164,61.342201,61.647385,...,761.284180,761.436768,730.155273,675.221985,607.165710,533.768677,459.761322,392.773193,343.638397,320.596924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,-5.340739,-2.441481,0.762963,5.645924,7.629627,4.730369,-3.509628,-16.174809,-28.382212,-34.333321,...,-103.457741,-27.314064,-12.512589,-61.037018,-168.309570,-318.308044,-487.380585,-652.485718,-792.413086,-887.478210
120,81.637009,80.263680,77.669601,76.601456,75.533310,73.549606,67.293312,63.631088,66.225159,69.887383,...,160.679947,226.905106,225.379181,152.745132,16.632587,-163.121429,-361.033936,-553.758301,-718.710876,-835.138977
121,156.559952,154.881424,154.271057,151.982162,150.761429,145.573288,138.706619,129.703659,127.414772,127.262177,...,916.013000,1017.181885,1055.024780,1027.252930,937.833740,801.110840,637.836792,470.900574,326.090271,230.872513
122,90.029602,84.383675,78.432564,75.228119,71.413307,63.936275,53.407391,44.099243,38.605911,35.096283,...,533.768677,623.340515,654.774597,624.408691,530.106506,378.124329,184.636978,-25.330362,-216.834000,-349.589508


In [4]:
string = "26,28,36,20,32,21,22,40,41,54,19,31,39,47,48,52,56,27,29,34,35,43,49,50,53,18,33,44,30,38,51,37,108,109,107,102,112,55,45,23"
string = [int(el) for el in string.split(",")]
eeg_data_df_filtered = in_df[
    (in_df["electrode"].isin(string))]
# type(string)
# string
eeg_data_df_filtered

,electrode,ms,value
18,18,0,2.288888
19,19,0,90.639969
20,20,0,-19.684437
21,21,0,-26.398510
22,22,0,-18.463697
...,...,...,...
63868906,102,515071,-656.300537
63868911,107,515071,-411.999847
63868912,108,515071,367.290253
63868913,109,515071,-320.749512


In [7]:
propogation_df = pd.read_json("/Users/siyuanzhao/Documents/DATADIR/patients/ep129/sample1/ep129_sorted_data.json")
propogation_test_df = propogation_df[propogation_df["eventIndex"] == 1]
propogation_test_df

,eventIndex,times,electrodes,network
0,1,"[103, 103, 104, 104, 104, 104, 104, 106, 106, ...","[18, 19, 20, 21, 22, 26, 27, 28, 29, 30, 31, 3...","[{'source': 26, 'target': 18}, {'source': 27, ..."
